In [13]:
from vpython import canvas, helix, rate, sphere, vector

In [41]:
def animate_spring(t_max, spring_length=1.0):
    canvas()
    pivot = sphere(pos=vector(0.0, 0.0, 0.0), radius=0.1)
    mass = sphere(pos=vector(0.0, -spring_length, 0.0), radius=0.2)
    mass.v = vector(0.0, 0.0, 0.0)
    spring = helix(pos=pivot.pos - vector(0.0, pivot.radius, 0.0),
                   axis=mass.pos - pivot.pos + vector(0.0, mass.radius, 0.0),
                   radius=0.1)
    g = 9.81
    k = 10.0
    t = 0.0
    delta_t = 0.01
    while t <= t_max:
        rate(int(1.0/delta_t))
        mass.v += delta_t*vector(0.0, (-g - k*(spring_length - spring.length)), 0.0)
        mass.pos += delta_t*mass.v
        spring.axis = mass.pos - pivot.pos + vector(0.0, mass.radius, 0.0)
        t += delta_t
        print(f'{mass.pos}, {mass.v}')

In [42]:
animate_spring(10.0)

<IPython.core.display.Javascript object>

<0, -1.00118, 0>, <0, -0.1181, 0>
<0, -1.00354, 0>, <0, -0.236082, 0>
<0, -1.00708, 0>, <0, -0.353828, 0>
<0, -1.01179, 0>, <0, -0.47122, 0>
<0, -1.01767, 0>, <0, -0.58814, 0>
<0, -1.02472, 0>, <0, -0.704473, 0>
<0, -1.03292, 0>, <0, -0.820101, 0>
<0, -1.04227, 0>, <0, -0.934909, 0>
<0, -1.05276, 0>, <0, -1.04878, 0>
<0, -1.06437, 0>, <0, -1.16161, 0>
<0, -1.07711, 0>, <0, -1.27327, 0>
<0, -1.09094, 0>, <0, -1.38366, 0>
<0, -1.10587, 0>, <0, -1.49266, 0>
<0, -1.12187, 0>, <0, -1.60018, 0>
<0, -1.13893, 0>, <0, -1.70609, 0>
<0, -1.15703, 0>, <0, -1.8103, 0>
<0, -1.17616, 0>, <0, -1.91269, 0>
<0, -1.19629, 0>, <0, -2.01318, 0>
<0, -1.21741, 0>, <0, -2.11165, 0>
<0, -1.23949, 0>, <0, -2.20801, 0>
<0, -1.26251, 0>, <0, -2.30216, 0>
<0, -1.28645, 0>, <0, -2.39401, 0>
<0, -1.31129, 0>, <0, -2.48346, 0>
<0, -1.33699, 0>, <0, -2.57043, 0>
<0, -1.36354, 0>, <0, -2.65484, 0>
<0, -1.3909, 0>, <0, -2.73658, 0>
<0, -1.41906, 0>, <0, -2.81559, 0>
<0, -1.44798, 0>, <0, -2.89179, 0>
<0, -1.47763, 0>, 